# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20901990


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/33 [00:00<?, ?it/s]

Rendering models:   6%|▌         | 2/33 [00:04<01:06,  2.14s/it]

Rendering models:   9%|▉         | 3/33 [00:07<01:12,  2.43s/it]

Rendering models:  12%|█▏        | 4/33 [00:07<00:52,  1.81s/it]

Rendering models:  15%|█▌        | 5/33 [00:07<00:36,  1.31s/it]

...

Rendering models:  88%|████████▊ | 29/33 [00:16<00:01,  2.16it/s]

Rendering models:  91%|█████████ | 30/33 [00:16<00:01,  2.50it/s]

Rendering models:  94%|█████████▍| 31/33 [00:17<00:00,  3.15it/s]

Rendering models:  97%|█████████▋| 32/33 [00:17<00:00,  3.04it/s]

equidad1                              0.147683
ElisabethB                            0.000825
not-logged-in-ab387df784182aa5a555    0.024277
not-logged-in-88f20bcb6fdeadfc1772    0.002964
john_mitchell_wv                      0.002455
not-logged-in-a924460f0f1f97422ce6    0.002777
esazure                               0.000560
not-logged-in-b505f80e181e4d9c3f50    0.012127
not-logged-in-60953d21f1d9f596b2fe    0.001275
iceman73                              0.003462
not-logged-in-204fc099209d1816fe0b    0.000755
not-logged-in-7e969a324309f376dde8    0.002794
not-logged-in-2ed1627faae6ff04b33d    0.003137
clars915                              0.000987
not-logged-in-5f30b2bb9b7f9d49da41    0.054678
Adrian_P                              0.021740
peakscience7                          0.000563
pangeli5                              0.000635
helios1235                            0.003265
Lavadude                              0.055050
Rian_NL                               0.000459
lord_oles    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())